## PRÁCTICA 1

Cargamos nuestro entorno

In [1]:
from pyspark import SparkContext
sc = SparkContext()


In [2]:
# DEFINIMOS PRIMERAMENTE NUESTRAS VARIABLES
#0 ESLA URL 
# 1 NO ES NADA 
#2 NO ES NADA 
#3 ES LA FECHA QUE HABRÁ QUE CONVERTIR, 
#4 NO LO SE 
#5 PROTOCOLO DE RESPUESTA: GET Y POST 
#6 URL DEL ARCHIVO
#7 Código de respuesta
#8 Http status
#9 Tamaño


In [3]:
import urllib
f = urllib.urlretrieve("https://www.dropbox.com/s/73wr8xb5s6fdj7g/apache.access.log.PROJECT?dl=1", "apache.access.log.PROJECT")

In [2]:
#Cargamos y definimos nuestro data
data_file = "./apache.access.log.PROJECT"
raw_data = sc.textFile(data_file)

Ya hemos cargado nuestro data, hemos intentado cargarlo con la ruta completa, pero el docker notiene acceso a 
Todas nuestras carpetas Con el small, pero como no podemos cargar un archivo de mas de 25mb lo cargamos de dropbx

In [3]:
##Contamos el numero de datos para explorar los datos
raw_data.count() 

1043177

In [6]:
##Realizamos la exploración de los datos
raw_data.take(5)

[u'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [7]:
##Ahora definimos nuestro archivo limpio, que seguira siendo un rdd

data_clean = raw_data.map(lambda x: x.replace(' -', ' 0')).map(lambda x: x.split(" "))

data_clean.take(1)


[[u'in24.inetnebr.com',
  u'0',
  u'0',
  u'[01/Aug/1995:00:00:01',
  u'00400]',
  u'"GET',
  u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  u'HTTP/1.0"',
  u'200',
  u'1839']]

EJERCICIO 1

In [8]:
##Calculamos el máximo 
data_clean.map(lambda x: int(x[-1])).max()


3421948

In [9]:
##Calculamos el mínimo
data_clean.map(lambda x: int(x[-1])).min()

0

In [10]:
##Calculamos la media

data_clean.map(lambda x: int(x[-1])).mean()

17531.55570243611

NUMERO DE PETICIONES POR CADA CODIGO DE RESPUESTA

In [11]:
##Ahora el ejercicio 2###############
# para el mismo se nos genera una tupla, todos los fallos con el número establecido

data_clean.map(lambda y: (int(y[-2]), 1)) 

PythonRDD[8] at RDD at PythonRDD.scala:48

In [12]:
##Importamos las librerias necesarias.
from operator import add


In [13]:
##Y ahora aplizamos el mapreduce, y que nos lo ordene de manera ascendente en función de su key que e
cod_response =data_clean.map(lambda y: (int(y[-2]), 1)).reduceByKey(add).sortByKey(ascending= True)

In [14]:
cod_response.take(5)

[(200, 940847), (302, 16244), (304, 79824), (403, 58), (404, 6185)]

EJERCICIO 3
- Mostrar 20 hosts que han sido visitados mas de 10 veces
- Definimos la vaiable sin grupar bajo el critero de mapeo de la primera columna

In [15]:
##Para el EJERCICIO 3 vamos a aplicar un filter, mostrar los 20 hosts, visitados mas de 10 veces

In [16]:

hosts = data_clean.map(lambda y: (y[0], 1))
#ahora realizamos la reducción, y ordenación, y por último un filtro sobre la 2 columna que es la de conteo, ya que el filter
#Se aplica a el nuevo rdd creado por el map.
agrupation = hosts.reduceByKey(add).sortByKey().filter(lambda x: x[-1] > 10)



In [17]:
##Ahora realizamos la agrupación de 20 host con mas de 10 visitas.
agrupation.take(20)

[(u'***.novo.dk', 16),
 (u'03-dynamic-c.wokingham.luna.net', 39),
 (u'05-dynamic-c.wokingham.luna.net', 25),
 (u'0875pr3e.pro.rpslmc.edu', 16),
 (u'1.ts1.mnet.medstroms.se', 32),
 (u'10.ts2.mnet.medstroms.se', 15),
 (u'101.irri.cgiar.org', 17),
 (u'1032015.ksc.nasa.gov', 90),
 (u'10forward.ppp.america.com', 18),
 (u'10md423.uni-duisburg.de', 39),
 (u'118.32.med.umich.edu', 15),
 (u'12.ts2.mnet.medstroms.se', 25),
 (u'120.33.med.umich.edu', 14),
 (u'128.100.180.37', 15),
 (u'128.100.183.252', 13),
 (u'128.100.183.253', 36),
 (u'128.100.191.43', 31),
 (u'128.100.197.90', 16),
 (u'128.100.8.177', 40),
 (u'128.101.13.25', 15)]

Ejercicio 4
- Encuentra los 10 endpoints maximos

In [18]:

endpoint = data_clean.map(lambda y: (y[6], 1))

In [19]:
##Aplicamos el reduce, luego lo ordenamos en el 2 valor de la tupla que es el contador, que nos lo ponga en orden descendiente, y que nos saque los 10 primeros.

top10endpoints = endpoint.reduceByKey(add).sortBy(lambda x: x[1], ascending = False).take(10)
top10endpoints

[(u'/images/NASA-logosmall.gif', 59666),
 (u'/images/KSC-logosmall.gif', 50420),
 (u'/images/MOSAIC-logosmall.gif', 43831),
 (u'/images/USA-logosmall.gif', 43604),
 (u'/images/WORLD-logosmall.gif', 43217),
 (u'/images/ksclogo-medium.gif', 41267),
 (u'/ksc.html', 28536),
 (u'/history/apollo/images/apollo-logo1.gif', 26766),
 (u'/images/launch-logo.gif', 24742),
 (u'/', 20175)]

Ejercicio 5
- Top 10 de endpoints visitados sin el código de respuesta 200

In [20]:
endpointno200 = data_clean.map(lambda y: (y[6], int(y[-2])))

In [21]:

no_200 = endpointno200.filter(lambda x: x[1] != 200).take(10)
no_200

[(u'/', 304),
 (u'/images/ksclogo-medium.gif', 304),
 (u'/images/MOSAIC-logosmall.gif', 304),
 (u'/images/USA-logosmall.gif', 304),
 (u'/images/WORLD-logosmall.gif', 304),
 (u'/images/NASA-logosmall.gif', 304),
 (u'/images/NASA-logosmall.gif', 304),
 (u'/images/MOSAIC-logosmall.gif', 304),
 (u'/images/USA-logosmall.gif', 304),
 (u'/images/WORLD-logosmall.gif', 304)]

EJERCICIO 6 
- Este es el número de hosts distintos

In [22]:
data_clean.map(lambda y: (y[0])).distinct().count()

54507

In [23]:
##Carga del nuevo contexto SQL
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

EJERCICIO 7
- Calcular el numero de host unicos cada dia

In [24]:
row_data = data_clean.map(lambda p: Row(host = p[0], UserId = int(p[1]), empty = p[2],
    fechas = p[3],get_post = p[4],endpoint = p[6],version = p[7],status_Code = int(p[-2]),size = p[-1]))

interactions_df = sqlContext.createDataFrame(row_data)

newdata_clean = interactions_df.withColumn('fechas', regexp_replace('fechas', '\[', ''))

newdata_clean2 = newdata_clean.withColumn('fechas', regexp_replace('fechas', 'Aug', '08'))

newdata_clean3 = newdata_clean2.select(from_unixtime(unix_timestamp('fechas', 'dd/MM/yyyy:hh:mm:ss')).alias('fechas'), 'host')

newdata_clean4 = newdata_clean3.na.fill(0)

#Muestra los host distintos de cada día del mes de Agosto
newdata_clean4.groupby(dayofmonth("fechas").alias("Dia del mes")).agg(countDistinct('host').alias('Host distintos')).sort('Dia del mes')\
        .show()

+-----------+--------------+
|Dia del mes|Host distintos|
+-----------+--------------+
|          1|          2582|
|          3|          3222|
|          4|          4190|
|          5|          2502|
|          6|          2537|
|          7|          4106|
|          8|          4406|
|          9|          4317|
|         10|          4523|
|         11|          4346|
|         12|          2864|
|         13|          2650|
|         14|          4454|
|         15|          4214|
|         16|          4340|
|         17|          4385|
|         18|          4168|
|         19|          2550|
|         20|          2560|
|         21|          4134|
+-----------+--------------+
only showing top 20 rows



Ejercicio 8
- Peticiones diarias por host

In [25]:
##### EJERCICIO 8 #####
# Calcular la media de peticiones diarias por host

parsed_df2 = newdata_clean4.withColumn("month", month(col("fechas"))).\
                                   withColumn("DayOfmonth", dayofmonth(col("fechas")))
parsed_df2.createOrReplaceTempView('parsed_df2')
sqlContext.sql("SELECT DATE(fechas) Date, COUNT(host)/COUNT(DISTINCT host)\
                AS Peticiones_diarias_host FROM parsed_df2 GROUP BY  DATE(fechas) ORDER BY DATE(fechas) ASC").show(n = 10)

+----------+-----------------------+
|      Date|Peticiones_diarias_host|
+----------+-----------------------+
|1995-08-01|     13.166537567776917|
|1995-08-03|     12.845127250155183|
|1995-08-04|     14.213365155131266|
|1995-08-05|     12.745003996802557|
|1995-08-06|     12.777296018919984|
|1995-08-07|     13.968582562104238|
|1995-08-08|     13.650022696323196|
|1995-08-09|      14.00440120454019|
|1995-08-10|     13.540791510059695|
|1995-08-11|     14.091578462954441|
+----------+-----------------------+
only showing top 10 rows



EJERCICIO 9
- Ahora 40 endpoints distintos que generen codigos de respuesta 404 

In [26]:
code400 = endpointno200.filter(lambda x: x[1] == 404).distinct().take(40)
code400

[(u'/shuttle/missions/mission.html', 404),
 (u'/space.pub.gif', 404),
 (u'/history/apollo.html', 404),
 (u'/msfe/astro_home.html', 404),
 (u'/shuttle/missions/missions,html', 404),
 (u'/shuttle/technology/sts-newref/stsref-toc.html', 404),
 (u'/history/apollo/sa-5/images/', 404),
 (u'/shuttle/technology/images/sts-comm.jpg', 404),
 (u'/history/appollo/apollo-13', 404),
 (u'/software/winvn.winvn.html', 404),
 (u'/hmhome.hmtl', 404),
 (u'/images/NASAlogo-small.gif', 404),
 (u'/history/apollo/aplool-13/apollo-13.html', 404),
 (u'/procurement/rschhdbk.html', 404),
 (u'/shuttle/missions/sts-70/images/ksc-95ec-o667.gif"', 404),
 (u'/cinaplex', 404),
 (u'/shuttle/resources/orbiters/atlantis.gif', 404),
 (u'//www.msfc.nasa.gov', 404),
 (u'/review.com/index.html', 404),
 (u'/history/apollo/pad-abort-test-1/docs/', 404),
 (u'/shuttle/technology/sts-newsref/sts-toc.html', 404),
 (u'/learning/museum/', 404),
 (u'/welcome/html', 404),
 (u'/shuttle/missions/sts-87/mission-sts-87.html', 404),
 (u'/sh

EJERCICIO 10

In [27]:
only404 = data_clean.filter(lambda x: int(x[-2]) == 404)

In [28]:
only404.map(lambda y: (y[6], 1)).reduceByKey(add).sortBy(lambda x: x[1], ascending = False).take(25)

[(u'/pub/winvn/readme.txt', 633),
 (u'/pub/winvn/release.txt', 494),
 (u'/shuttle/missions/STS-69/mission-STS-69.html', 430),
 (u'/images/nasa-logo.gif', 319),
 (u'/elv/DELTA/uncons.htm', 178),
 (u'/shuttle/missions/sts-68/ksc-upclose.gif', 154),
 (u'/history/apollo/sa-1/sa-1-patch-small.gif', 146),
 (u'/images/crawlerway-logo.gif', 120),
 (u'/://spacelink.msfc.nasa.gov', 117),
 (u'/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif', 100),
 (u'/history/apollo/a-001/a-001-patch-small.gif', 97),
 (u'/images/Nasa-logo.gif', 85),
 (u'/shuttle/resources/orbiters/atlantis.gif', 63),
 (u'/history/apollo/images/little-joe.jpg', 62),
 (u'/images/lf-logo.gif', 59),
 (u'/shuttle/resources/orbiters/discovery.gif', 56),
 (u'/shuttle/resources/orbiters/challenger.gif', 54),
 (u'/robots.txt', 53),
 (u'/elv/new01.gif>', 43),
 (u'/history/apollo/pad-abort-test-2/pad-abort-test-2-patch-small.gif', 38),
 (u'/pub', 36),
 (u'/pub/', 36),
 (u'/history/apollo/sa-2/sa-2-patch-small.gif', 35),
 

EJERCICIO 11
- Top 5 días que generaron código de respuesta 404

In [29]:
newdata_clean5 = newdata_clean2.select(from_unixtime(unix_timestamp('fechas', 'dd/MM/yyyy:hh:mm:ss')).alias('fechas'), 'status_Code')

newdata_clean5.filter(newdata_clean5['status_Code'] == 404).groupBy(dayofmonth('fechas')).count().sort(col("count").desc()).show(n = 5)


+------------------+-----+
|dayofmonth(fechas)|count|
+------------------+-----+
|                 7|  532|
|                 8|  381|
|                 6|  372|
|                 4|  346|
|                15|  326|
+------------------+-----+
only showing top 5 rows

